# Processing sentinel-2 data

## DATA MANAGMENT and PREPARATION

In [1]:
import os
from glob import glob
import zipfile
import shutil
import more_itertools
from more_itertools import unique_everseen

# import gui tkinter
from tkinter import Tk
import tkinter.filedialog as tkfd

import logging

In [2]:
logging.basicConfig(format='%(message)s', level='INFO')

### Set input and output directories

A **tkinter GUI** is loaded:
- to locate and select the **input zipfiles**,
- to define the **output directory**, and
- to locate a GeoJSON file, defining the **bouding box of Flanders**.

The tkinter input is printed to the console.

In [3]:
# tkinter GUI
root = Tk()
input_zip_files = tkfd.askopenfilenames(title = "Select input dataset (zipfiles)", 
                                             initialdir = "C:/", 
                                             filetypes = (("zipfile", "*.zip"),("all files", "*.*")))
output_dir = tkfd.askdirectory(title = "Select output folder", initialdir = "C:/")
bounding_box_flanders = tkfd.askopenfilename(title = "Select input GeoJSON, defining the bounding box of Flanders", 
                                             initialdir = "C:/", 
                                             filetypes = (("GeoJSON", "*.geojson"),("all files", "*.*")))
root.destroy()

# print statements: the tkinter input is printed to the console
print("The following files have been selected for processing:")
for zip_file in input_zip_files:
    print("-", zip_file)
print(" ")
print("The selected output folder is:")
print("-", output_dir)
print(" ")
print("The selected bounding box is located at:")
print("-", bounding_box_flanders)

The following files have been selected for processing:
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip
 
The selected output folder is:
- C:/SS
 
The selected bounding box is located at:
- C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson


### REVIEW of the selected input zipfiles

The output directory is checked for what has been done in the past. Files with a date that already exists in the output directory will be removed from the selection. An overview is printed to the console.

In [4]:
validated_zip_files = []
to_process = 0
to_process_dates = []
no_action = 0
no_action_dates = []

print("The input zip files are validated:")
for input_zip_file in input_zip_files:
    date = input_zip_file[-19:-11]
    if date not in os.listdir(output_dir):
        validated_zip_files.append(input_zip_file)
        to_process = to_process + 1
        to_process_dates.append(date)
        print("-", input_zip_file, "will be processed.")
    else:
        no_action = no_action + 1
        no_action_dates.append(date)
        print("- WARNING: An output folder with date", date, "already exist.", input_zip_file, "is dropped from the selection.")
        
to_process_dates = list(unique_everseen(to_process_dates))
no_action_dates = list(unique_everseen(no_action_dates))

# print statements
print(" ")
print("SUMMARY:")
print("---")
print("The following date(s) already exist in the output folder:")
if no_action == 0:
    print("- None")
else:
    for no_action_date in no_action_dates:
        print("-", no_action_date)
print("The corresponding", no_action, "file(s) were dropped from the selection.")
print(" ")
print("File(s) with the following dates will be processed:")
if to_process == 0:
    print("- None")
else:
    for to_process_date in to_process_dates:
        print("-", to_process_date)
print("The corresponding", to_process, "file(s) will be processed:")        
for validated_zip_file in validated_zip_files:
    print("-", validated_zip_file)

The input zip files are validated:
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip will be processed.
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip will be processed.
 
SUMMARY:
---
The following date(s) already exist in the output folder:
- None
The corresponding 0 file(s) were dropped from the selection.
 
File(s) with the following dates will be processed:
- 20160826
The corresponding 2 file(s) will be processed:
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip


### UNZIP the data

The sentinel data is donwloaded and stored as a zipfile. A temporary folder is created in the output directory, named "tempUZ" where the zipfiles will be unpacked and accessed for processing.

In [5]:
# A temporary folder is created in the output directory, named "tempUZ", where the files will be unzipped.
extraction_zip_dir = output_dir + "/tempUZ/"
if not os.path.isdir(output_dir + "/tempUZ/"):
    os.mkdir(output_dir + "/tempUZ/")
    
# unzip files
for validated_zip_file in validated_zip_files:
    print("Extracting", validated_zip_file, "started...")
    z = zipfile.ZipFile(validated_zip_file)
    z.extractall(extraction_zip_dir)
    print("Extracting", validated_zip_file, "done!")

# print statement
print("All datasets unpacked. Unzipping finished!")

Extracting C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip started...
Extracting C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip done!
Extracting C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip started...
Extracting C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip done!
All datasets unpacked. Unzipping finished!


### DELETE useless data

Only granules within or partially within the bounding box of Flanders are kept. All other granules are deleted.

Eight granules overlap with the Flanders bounding box: 
* T31UDT - T31UET - T31UFT - T31UGT
* T31UDS - T31UES - T31UFS - T31UGS


In [6]:
print("Start data management procedure...")
# Eight granules overlap with the Flanders bounding box
selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
# select folders with granules overlapping the bounding box of Flanders...
selected_folders =[]
for folder in glob(os.path.join(extraction_zip_dir, "**", "*/GRANULE/*"), recursive=True):
    for granule in selected_granules:
        if granule in folder:
            selected_folders.append(folder)
# and delete the other granules.
for folder in glob(os.path.join(extraction_zip_dir, "**", "*/GRANULE/*"), recursive=True):
    if folder not in selected_folders:
        shutil.rmtree(folder)

Start data management procedure...


### REARRANGE data

The data is rearranged and grouped by date to easily allow mosaicing of imagery. This happens in three steps:
- A temporary "processing" folder, named "P", is created. 
- The data is grouped by date and moved to the processing folder "P".
- The now empty temporary zip directory "tempUZ" is deleted.

In [7]:
# A temporary "processing" folder, named "P", is created. 
processing_dir = output_dir + "/" + "P"
if not os.path.isdir(processing_dir):
    os.mkdir(processing_dir)
print("Processing directory", processing_dir, "created.")
    
# The data is grouped by date and moved to the processing folder.
for filename in os.listdir(extraction_zip_dir):
    date = filename[-20:-12]
    date_folder = processing_dir + "/" + date
    if not os.path.isdir(date_folder):
        os.mkdir(date_folder)
    folder = extraction_zip_dir + "/" + filename
    shutil.move(src = folder, dst = date_folder)

print("Data moved to", processing_dir)
    
# The now empty temporary zip directory "tempUZ" is deleted.
shutil.rmtree(extraction_zip_dir)

print("Data management procedure finished.")

Processing directory C:/SS/P created.
Data moved to C:/SS/P
Data management procedure finished.


## DATA PROCESSING

In [8]:
import rsgislib
from rsgislib import imageutils, RSGISPyUtils
import osgeo
from osgeo import gdal, osr, ogr
from rasterstats import zonal_stats
import subprocess
import numpy as np
from numpy import *

GDAL data files not located, GDAL_DATA not set
PROJ data files not located, PROJ_LIB not set


In [9]:
# Enable GDAL/OGR exceptions and set GDAL data
gdal.UseExceptions()
osgeo.gdal.SetConfigOption("GDAL_DATA", "C:/Users/jeroen.dereu@inbo.be/Software/gdal-2.1.3/data")

In [10]:
#listdir checken
folders_to_be_processed = []
for date_folder in os.listdir(processing_dir): 
    folders_to_be_processed.append(processing_dir + "/" + date_folder + "/")

print("The following folders will be processed:")
for date_folder in folders_to_be_processed:
    print("-", date_folder)

The following folders will be processed:
- C:/SS/P/20160826/


### CONVERSION: JP2 to GeoTiff

Images are converted from the JP2 to the GeoTiff fileformat.
- Images are converted one by one. This means one input image results in one output image.
- Coordinates are transformed from EPSG:32631 to EPSG:4326.
- The Geotiffs are written to a new subfolder, named "ind_tifs".
- The original filename is preserved.

In [11]:
for date_folder in folders_to_be_processed:
    print("Start converting imagery in", date_folder, ": JP2 to GeoTiff...")
    for path in glob(os.path.join(date_folder, "**", "*.jp2"), recursive=True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        if not os.path.isdir(date_folder + "/ind_tifs"):
            os.mkdir(date_folder + "/ind_tifs")
        output_folder = date_folder + "/ind_tifs/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  dstSRS = "EPSG:4326", 
                  srcSRS = "EPSG:32631")
    print("Finished converting imagery in", date_folder, ": JP2 to GeoTiff!")
print("All datasets converted. Conversion from JP2 to GeoTiff finished!")

Start converting imagery in C:/SS/P/20160826/ : JP2 to GeoTiff...
Finished converting imagery in C:/SS/P/20160826/ : JP2 to GeoTiff!
All datasets converted. Conversion from JP2 to GeoTiff finished!


### MOSAIC individual bands

A mosaic of the available imagery per day for each of the 13 bands is created. Also one overview mosaic is produced.
- For each day, 14 images (GeoTiff) are created.
- The output coordinatesystem is EPSG:4326.
- The mosaic is cropped to the bounding box Flanders.
- The mosaics are written to a new subfolder, named "merged_bands".
- The filename consists of the date and the bandnumber (e.g. 20160720_B05.tif).



                        Detailed overview of the 13 bands.
|Band name	|Resolution (m)	|Central wavelength (nm)	|Band width (nm)	|Purpose
|:---:|:---:|:---:|:---:|:---:|
|B01	|60	|443	|20	|Aerosol detection|
|B02	|10	|490	|65	|Blue|
|B03	|10	|560	|35	|Green|
|B04	|10	|665	|30	|Red|
|B05	|20	|705	|15	|Vegetation classification|
|B06	|20	|740	|15	|Vegetation classification|
|B07	|20	|783	|20	|Vegetation classification|
|B08	|10	|842	|115	|Near infrared|
|B08A	|20	|865	|20	|Vegetation classification|
|B09	|60	|945	|20	|Water vapour|
|B10	|60	|1375	|30	|Cirrus|
|B11	|20	|1610	|90	|Snow / ice / cloud discrimination|
|B12	|20	|2190	|180	|Snow / ice / cloud discrimination|

In [12]:
for date_folder in folders_to_be_processed:
    print("Start mosaicing bands in", date_folder, "...")
    date = date_folder[-9:-1]
    selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
    selected_images = []
    B01 = []
    B02 = []
    B03 = []
    B04 = []
    B05 = []
    B06 = []
    B07 = []
    B08 = []
    B8A = []
    B09 = []
    B10 = []
    B11 = []
    B12 = []
    overview = []
    for path in glob(os.path.join(date_folder, "**", "*.jp2"), recursive=True):
        for granule in selected_granules:
            if granule in path:
                selected_images.append(path)
    for image in selected_images:
        if image.endswith("B01.jp2"):
            B01.append(image)
        elif image.endswith("B02.jp2"):
            B02.append(image)
        elif image.endswith("B03.jp2"):
            B03.append(image)
        elif image.endswith("B04.jp2"):
            B04.append(image)
        elif image.endswith("B05.jp2"):
            B05.append(image)
        elif image.endswith("B06.jp2"):
            B06.append(image)
        elif image.endswith("B07.jp2"):
            B07.append(image)
        elif image.endswith("B08.jp2"):
            B08.append(image)        
        elif image.endswith("B8A.jp2"):
            B8A.append(image)
        elif image.endswith("B09.jp2"):
            B09.append(image)
        elif image.endswith("B10.jp2"):
            B10.append(image)        
        elif image.endswith("B11.jp2"):
            B11.append(image)      
        elif image.endswith("B12.jp2"):
            B12.append(image)         
        else:
            overview.append(image)
    if not os.path.isdir(date_folder + "/merged_bands"):
        os.mkdir(date_folder + "/merged_bands")
    output_folder = date_folder + "/merged_bands/"
    output_B01 = output_folder + date + "_B01" + ".tif"
    output_B02 = output_folder + date + "_B02" + ".tif"
    output_B03 = output_folder + date + "_B03" + ".tif"
    output_B04 = output_folder + date + "_B04" + ".tif"
    output_B05 = output_folder + date + "_B05" + ".tif"
    output_B06 = output_folder + date + "_B06" + ".tif"
    output_B07 = output_folder + date + "_B07" + ".tif"
    output_B08 = output_folder + date + "_B08" + ".tif"
    output_B8A = output_folder + date + "_B8A" + ".tif"
    output_B09 = output_folder + date + "_B09" + ".tif"   
    output_B10 = output_folder + date + "_B10" + ".tif"
    output_B11 = output_folder + date + "_B11" + ".tif"
    output_B12 = output_folder + date + "_B12" + ".tif"
    output_overview = output_folder + date + "_overview" + ".tif"
    gdal.Warp(destNameOrDestDS = output_B01, 
              srcDSOrSrcDSTab = B01, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B02, 
              srcDSOrSrcDSTab = B02, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B03, 
              srcDSOrSrcDSTab = B03, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)        
    gdal.Warp(destNameOrDestDS = output_B04, 
              srcDSOrSrcDSTab = B04, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)           
    gdal.Warp(destNameOrDestDS = output_B05, 
              srcDSOrSrcDSTab = B05, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)             
    gdal.Warp(destNameOrDestDS = output_B06, 
              srcDSOrSrcDSTab = B06, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B07, 
              srcDSOrSrcDSTab = B07, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B08, 
              srcDSOrSrcDSTab = B08, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B8A, 
              srcDSOrSrcDSTab = B8A, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B09, 
              srcDSOrSrcDSTab = B09, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B10, 
              srcDSOrSrcDSTab = B10, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B11, 
              srcDSOrSrcDSTab = B11, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B12, 
              srcDSOrSrcDSTab = B12, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_overview, 
              srcDSOrSrcDSTab = overview, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    print("Finished mosaicing bands in", date_folder, "!")
print("All datasets mosaiced. Mosaicing finished!")

Start mosaicing bands in C:/SS/P/20160826/ ...
Finished mosaicing bands in C:/SS/P/20160826/ !
All datasets mosaiced. Mosaicing finished!


### NO DATA check

In some cases, it appears that there is no data within the Flanders bounding box. Further processing of these datasets is useless. At this stage, the processed data is checked for empty datasets. Empty datasets will be removed and excluded from further processing.

In [13]:
print("The processed data is checked for empty datasets. Empty datasets will be removed and excluded from further processing.")
print(" ")
folders_to_remove = []
remove_count = 0
for date_folder in folders_to_be_processed:
    for path in glob(os.path.join(date_folder, "*merged_bands*", "*B01.tif"), recursive=True):
        mean = zonal_stats(bounding_box_flanders, path, stats = "mean", nodata = 0)
        if mean != [{'mean': None}]:
            pass
        else:
            folders_to_be_processed.remove(date_folder)
            folders_to_remove.append(date_folder)
            remove_count = remove_count + 1
            print("Imagery in", date_folder, "contains no data.", date_folder, "will be exluded from further processing and the empty data will removed from the output directory.")

for folder_to_remove in folders_to_remove:
    shutil.rmtree(folder_to_remove)
    
# print statements    
print(" ")
print("SUMMARY")
print("---")
print(remove_count, "dataset(s) have been deleted!")
print("---")
print("Processing will continue with the following dataset(s)")
if folders_to_be_processed == []:
    print("- None")
else:
    for folder_to_be_processed in folders_to_be_processed:
        print("-", folder_to_be_processed)

GDAL data files not located, GDAL_DATA not set
PROJ data files not located, PROJ_LIB not set
GDAL data files not located, GDAL_DATA not set
PROJ data files not located, PROJ_LIB not set


The processed data is checked for empty datasets. Empty datasets will be removed and excluded from further processing.
 
 
SUMMARY
---
0 dataset(s) have been deleted!
---
Processing will continue with the following dataset(s)
- C:/SS/P/20160826/


### RGB-image

An RGB (Red-Green_Blue) image is created by stacking band B02 (Blue), B03 (Green) and B04 (Red) into one GeoTiff.
- For each day, 1 RGB-images (GeoTiff) is created.
- The output coordinatesystem is EPSG:4326.
- The RGB image is written to a new subfolder, named "RGB".
- The filename consists of the date and "RGB" (e.g. 20160720_RGB.tif).

In [14]:
for date_folder in folders_to_be_processed:
    print("Start creating RGB-image for", date_folder, "...")
    RGB = []
    for path in glob(os.path.join(date_folder, "**", "merged_bands", "*.tif"), recursive=True):
        if path.endswith("B04.tif"):
            RGB.append(path)
        if path.endswith("B03.tif"):
            RGB.append(path) 
        if path.endswith("B02.tif"):
            RGB.append(path)
    bandNamesList = ["Red", "Green", "Blue"]
    if not os.path.isdir(date_folder + "/RGB/"):
        os.mkdir(date_folder + "/RGB/")
    date = date_folder[-9:-1]
    outputImage = date_folder + "/RGB/" + date + "_RGB.tif"
    gdalFormat = "GTiff"
    dataType = rsgislib.TYPE_16UINT
    imageutils.stackImageBands(RGB, bandNamesList, outputImage, None, 0, gdalFormat, dataType)
    print("Finished creating RGB-image for", date_folder, "!")
print("All RGB-images produced. RGB-imaging finished!")

Start creating RGB-image for C:/SS/P/20160826/ ...
Finished creating RGB-image for C:/SS/P/20160826/ !
All RGB-images produced. RGB-imaging finished!


### NDVI - Normalised Difference Vegetation Index

**NDVI = (NIR - R) / (NIR + R)**

    NIR: Near Infra Red - Sentinelsat band 8
    R: Visible Red - Sentinelsat band 4

An NDVI image is created by performing a raster calculation using B04 (Red) and B08 (NIR), (B08 - B04) / (B08 + B04),  and writting the output to a GeoTiff.
- For each day, 1 RGB-images (GeoTiff) is created.
- The output coordinatesystem is EPSG:4326.
- The NDVI image is written to a new subfolder, named "NDVI".
- The filename consists of the date and "NDVI" (e.g. 20160720_NDVI.tif).

In [15]:
for date_folder in folders_to_be_processed:
    print("Start creating NDVI-image for", date_folder, "...")
    for path in glob(os.path.join(date_folder, "**", "merged_bands", "*tif"), recursive=True):
        if path.endswith("B04.tif"):
            B04 = gdal.Open(path)
        if path.endswith("B08.tif"):
            B08 = gdal.Open(path)
    red = B04.ReadAsArray()
    nir = B08.ReadAsArray()
    check = np.logical_and(red > 0, nir > 0)
    ndvi = np.where(check, (nir - red) / (nir + red), -999)
    geo = B08.GetGeoTransform()
    proj = B08.GetProjection()
    shape = red.shape
    driver = gdal.GetDriverByName("GTiff")
    # define output filename and location
    if not os.path.isdir(date_folder + "/NDVI/"):
        os.mkdir(date_folder + "/NDVI/")
    date = date_folder[-9:-1]
    ndvi_file = date_folder + "/NDVI/" + date + "_ndvi.tif"
    # create and write the nvdi-file
    dst_ds = driver.Create(ndvi_file, shape[1], shape[0], 1, gdal.GDT_Float32)
    dst_ds.SetGeoTransform(geo)
    dst_ds.SetProjection(proj)
    dst_ds.GetRasterBand(1).WriteArray(ndvi)
    dst_ds = None
    B04 = None
    B08 = None
    print("Finished creating NDVI-image for", date_folder, "!")
print("All NDVI-images produced. NDVI-imaging finished!")

Start creating NDVI-image for C:/SS/P/20160826/ ...


C:\Users\jeroen.dereu@inbo.be\AppData\Local\Continuum\Anaconda3\envs\osgeoenv\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Finished creating NDVI-image for C:/SS/P/20160826/ !
All NDVI-images produced. NDVI-imaging finished!


### DATA MANAGEMENT

The last step is a data managment step:
- The raw data is removed from the directory, only the processing output is kept.
- The processed data is moved to the output directory.
- The now empty processing directory is removed.

In [16]:
print("Start data managment procedure...")
# The raw data is removed from the directory.
for path in glob(os.path.join(processing_dir, "**", "S2A*.SAFE"), recursive=True):
    shutil.rmtree(path)
    
# The processed data is moved to the output directory.
for processed_dataset in os.listdir(processing_dir):
    processed_dataset_folder = processing_dir + "/" + processed_dataset
    shutil.move(src = processed_dataset_folder, dst = output_dir)
print("All processing output moved to", output_dir)

# The now empty processing directory, named "P", is removed.
shutil.rmtree(processing_dir)

print("Data management procedure done!")
print("Check", output_dir, "for processing output.")
print("Process finished!")

Start data managment procedure...
All processing output moved to C:/SS
Data management procedure done!
Check C:/SS for processing output.
Process finished!
